<h1>Affordability</h1>

In [27]:
# Importing necessary libraries
import geopandas as gpd 
import folium
import pandas as pd
import sys

sys.path.insert(0, "../scripts/")
from helper_functions import convert_census_to_postcode

### Read and create relevant data

In [28]:
# Reading in preprocessed property data
property_df = pd.read_csv("../data/curated/categorised_distances.csv")
property_df = property_df[["Cost", "Bed", "Bath", "Parking", "Postcode"]]

In [29]:
median_rent_df = pd.DataFrame(property_df.groupby(["Postcode"])["Cost"].median()).reset_index()

### Calculating the average number of facilities per postcode

In [30]:
avg_bed = property_df.groupby(by = "Postcode")["Bed"].mean()
avg_bed.head()

Postcode
3000    1.601399
3002    1.909091
3003    1.796875
3004    1.771739
3006    1.785714
Name: Bed, dtype: float64

In [31]:
avg_bath = property_df.groupby(by = "Postcode")["Bath"].mean()
avg_bath.head()

Postcode
3000    1.300699
3002    1.318182
3003    1.343750
3004    1.423913
3006    1.469388
Name: Bath, dtype: float64

In [32]:
avg_parking = property_df.groupby(by = "Postcode")["Parking"].mean()
avg_parking.head()

Postcode
3000    0.356643
3002    1.000000
3003    0.609375
3004    1.097826
3006    0.647959
Name: Parking, dtype: float64

In [33]:
# Creating dataframe of average facilities per postcode
facilities_df = pd.DataFrame()
facilities_df["Postcode"] = property_df["Postcode"].unique()
facilities_df["Average # Beds"] = avg_bed.to_list()
facilities_df["Average # Baths"] = avg_bath.to_list()
facilities_df["Average # Parking"] = avg_parking.to_list()
facilities_df.head()

,Postcode,Average # Beds,Average # Baths,Average # Parking
0,3000,1.601399,1.300699,0.356643
1,3002,1.909091,1.318182,1.000000
2,3003,1.796875,1.343750,0.609375
3,3004,1.771739,1.423913,1.097826
4,3006,1.785714,1.469388,0.647959


In [34]:
# Joining median rental price per postcode with average facilities
facilities_df = facilities_df.set_index("Postcode")
price_by_facility_df = median_rent_df.join(facilities_df, on = "Postcode")
price_by_facility_df = price_by_facility_df.set_index("Postcode")
price_by_facility_df.head()

,Cost,Average # Beds,Average # Baths,Average # Parking
Postcode,,,,
3000,520.0,1.601399,1.300699,0.356643
3002,642.5,1.909091,1.318182,1.000000
3003,520.0,1.796875,1.343750,0.609375
3004,550.0,1.771739,1.423913,1.097826
3006,551.5,1.785714,1.469388,0.647959


### Obtaining census/income data

In [35]:
# Reading in csv files for obtaining census data
sa2_postcode_map = pd.read_csv("../data/curated/sa2_postcode_mapping_2021.csv")
sa2_postcode_map.set_index("sa2_2021", inplace = True)
census_df = pd.read_csv("../data/curated/census_data.csv")

In [36]:
# Using helper function to convert sa2 mapping to postcodes
census_by_postcode_df = convert_census_to_postcode(census_df, sa2_postcode_map, "mean_no_zero")
census_by_postcode_df.head()

,postcode_2021,tot_population_11,tot_population_16,tot_population_21,avg_med_mortg_rep_11,avg_med_mortg_rep_16,avg_med_mortg_rep_21,avg_med_person_inc_11,avg_med_person_inc_16,avg_med_person_inc_21,avg_med_rent_16,avg_med_rent_11,avg_med_rent_21,avg_med_hh_inc_16,avg_med_hh_inc_11,avg_med_hh_inc_21,tot_avg_hh_size_16,tot_avg_hh_size_11,tot_avg_hh_size_21
0,3000,124551,167166,178424,2213.38,2040.38,2040.19,862.18,5483.82,6467.76,395.76,447.06,418.19,1482.53,1896.76,2159.41,1.88,1.97,1.86
1,3002,68729,82804,89023,2357.78,2173.67,2155.22,1091.80,8969.60,10432.90,398.00,460.33,449.67,1709.40,2415.00,2598.80,1.82,1.91,1.87
2,3003,15496,20633,23083,2200.00,2050.00,2085.00,701.50,716.00,1000.00,395.00,418.50,385.50,1466.00,1493.50,1751.00,2.15,2.15,1.95
3,3004,100879,123254,129273,2331.58,2155.67,2149.75,1066.08,7152.46,8339.46,391.15,446.83,440.75,1688.85,2270.46,2471.46,1.83,1.89,1.84
4,3006,21150,30239,36164,2477.25,2217.75,2079.00,1132.40,16783.00,19507.00,406.80,501.00,461.00,1637.20,2883.20,3088.80,1.80,1.92,1.92


In [37]:
# Removing unnecessary columns from census data (only need columns containing data from 2021)
census_by_postcode_df = census_by_postcode_df[["postcode_2021", "tot_population_21", "avg_med_mortg_rep_21", "avg_med_person_inc_21", "avg_med_rent_21", 
                            "avg_med_hh_inc_21", "tot_avg_hh_size_21"]]

# Selecting columns required for assessing affordability
census_income_df = census_by_postcode_df[["postcode_2021", "avg_med_mortg_rep_21", "avg_med_hh_inc_21"]]
census_income_df.head()

,postcode_2021,avg_med_mortg_rep_21,avg_med_hh_inc_21
0,3000,2040.19,2159.41
1,3002,2155.22,2598.80
2,3003,2085.00,1751.00
3,3004,2149.75,2471.46
4,3006,2079.00,3088.80


In [38]:
# Renaming postcode column of census dataframe and setting to index
census_income_df = census_income_df.rename({"postcode_2021": "Postcode"}, axis = 1)
census_income_df = census_income_df.set_index("Postcode")
census_income_df.head()

,avg_med_mortg_rep_21,avg_med_hh_inc_21
Postcode,,
3000,2040.19,2159.41
3002,2155.22,2598.80
3003,2085.00,1751.00
3004,2149.75,2471.46
3006,2079.00,3088.80


In [39]:
# Joining census income dataframe to dataframe containg price & facilities information
df = census_income_df.join(price_by_facility_df, on = "Postcode", how = "right")
df.head()

,avg_med_mortg_rep_21,avg_med_hh_inc_21,Cost,Average # Beds,Average # Baths,Average # Parking
Postcode,,,,,,
3000,2040.19,2159.41,520.0,1.601399,1.300699,0.356643
3002,2155.22,2598.80,642.5,1.909091,1.318182,1.000000
3003,2085.00,1751.00,520.0,1.796875,1.343750,0.609375
3004,2149.75,2471.46,550.0,1.771739,1.423913,1.097826
3006,2079.00,3088.80,551.5,1.785714,1.469388,0.647959


### Calculating affordability metric

In [40]:
# If the number of facilities is 0 (i.e. for number of bedrooms or number of parking spaces) replace 0 value with arbitrary small value epsilon
EPSILON = 10**(-6)
 
df.loc[df["Average # Beds"] == 0, "Average # Beds"] = EPSILON
df.loc[df["Average # Parking"] == 0, "Average # Parking"] = EPSILON

In [41]:
# Calculating weights for facilities (from previous correlation matrix know positive correlation exists between all 3 attributes and the cost)
bed_weight = df["Average # Beds"].corr(df["Cost"])
bath_weight = df["Average # Baths"].corr(df["Cost"])
parking_weight = df["Average # Parking"].corr(df["Cost"])

In [42]:
# Taking the median rental price divided by the median income (multiply by 100 to get cost as a percentage of income)
df["(Cost / Income)%"] = (df["Cost"] / df["avg_med_hh_inc_21"]) * 100
df.head()

,avg_med_mortg_rep_21,avg_med_hh_inc_21,Cost,Average # Beds,Average # Baths,Average # Parking,(Cost / Income)%
Postcode,,,,,,,
3000,2040.19,2159.41,520.0,1.601399,1.300699,0.356643,24.080652
3002,2155.22,2598.80,642.5,1.909091,1.318182,1.000000,24.722949
3003,2085.00,1751.00,520.0,1.796875,1.343750,0.609375,29.697316
3004,2149.75,2471.46,550.0,1.771739,1.423913,1.097826,22.254052
3006,2079.00,3088.80,551.5,1.785714,1.469388,0.647959,17.854830


In [43]:
# Taking the median mortage divided by the median income (multiply by 100 to get mortage as a percentage of income)
df["(Mortage / Income)%"] = (df["avg_med_mortg_rep_21"] / df["avg_med_hh_inc_21"]) * 100
df.head()

,avg_med_mortg_rep_21,avg_med_hh_inc_21,Cost,Average # Beds,Average # Baths,Average # Parking,(Cost / Income)%,(Mortage / Income)%
Postcode,,,,,,,,
3000,2040.19,2159.41,520.0,1.601399,1.300699,0.356643,24.080652,94.479048
3002,2155.22,2598.80,642.5,1.909091,1.318182,1.000000,24.722949,82.931353
3003,2085.00,1751.00,520.0,1.796875,1.343750,0.609375,29.697316,119.074814
3004,2149.75,2471.46,550.0,1.771739,1.423913,1.097826,22.254052,86.982998
3006,2079.00,3088.80,551.5,1.785714,1.469388,0.647959,17.854830,67.307692


In [44]:
# Calculating non-stadardised affordability metric
df["Affordability"] = ((bed_weight * df["Average # Beds"]) + (bath_weight * df["Average # Baths"]) + (parking_weight * df["Average # Parking"]) + 
                        df["(Cost / Income)%"] + df["(Mortage / Income)%"])
df.head()

,avg_med_mortg_rep_21,avg_med_hh_inc_21,Cost,Average # Beds,Average # Baths,Average # Parking,(Cost / Income)%,(Mortage / Income)%,Affordability
Postcode,,,,,,,,,
3000,2040.19,2159.41,520.0,1.601399,1.300699,0.356643,24.080652,94.479048,119.799725
3002,2155.22,2598.80,642.5,1.909091,1.318182,1.000000,24.722949,82.931353,109.175913
3003,2085.00,1751.00,520.0,1.796875,1.343750,0.609375,29.697316,119.074814,150.165416
3004,2149.75,2471.46,550.0,1.771739,1.423913,1.097826,22.254052,86.982998,110.781955
3006,2079.00,3088.80,551.5,1.785714,1.469388,0.647959,17.854830,67.307692,86.617690


In [45]:
# Calculating standardised affordability metric
min_affordability = df.sort_values(by = "Affordability").head(1)["Affordability"].tolist()[0]
max_affordability = df.sort_values(by = "Affordability", ascending = False).head(1)["Affordability"].tolist()[0]

df["Standardised Affordability"] = (df["Affordability"] - min_affordability) / (max_affordability - min_affordability)

# Taking 1 - standardised affordability (to make sure metric ranks 0 as least affordable & 1 as most affordable)
df["Standardised Affordability"] = (1 - df["Standardised Affordability"])

# Printing the top 10 most affordable postcodes
df.sort_values(by = "Standardised Affordability", ascending = False).head(10)

,avg_med_mortg_rep_21,avg_med_hh_inc_21,Cost,Average # Beds,Average # Baths,Average # Parking,(Cost / Income)%,(Mortage / Income)%,Affordability,Standardised Affordability
Postcode,,,,,,,,,,
3424,714.33,1122.33,115.0,0.500000,1.000000,1.000000,10.246541,63.647056,74.772529,1.000000
3409,946.00,1371.50,200.0,1.000000,1.000000,1.000000,14.582574,68.975574,84.614814,0.947093
3414,737.00,1179.00,245.0,2.000000,1.000000,1.000000,20.780322,62.510602,84.703058,0.946619
3418,737.00,1179.00,250.0,2.000000,1.000000,2.000000,21.204411,62.510602,85.382711,0.942966
3006,2079.00,3088.80,551.5,1.785714,1.469388,0.647959,17.854830,67.307692,86.617690,0.936327
3396,714.33,1122.33,255.0,2.500000,1.000000,1.000000,22.720590,63.647056,87.957513,0.929125
3393,714.33,1122.33,295.0,3.000000,1.000000,1.000000,26.284604,63.647056,91.699261,0.909012
3490,900.00,1239.00,220.0,3.000000,1.000000,1.000000,17.756255,72.639225,92.163081,0.906518
3390,903.00,1333.00,300.0,3.000000,1.000000,2.000000,22.505626,67.741935,92.270727,0.905940


In [46]:
df = df.reset_index()
affordability_df = df[["Postcode", "Standardised Affordability"]]

### Graphing standardised affordability

In [47]:
# Creating geoJSON file of postcode and geometry coordinates 
sf = gpd.read_file("../data/raw/external/POA_2021_AUST_GDA2020/POA_2021_AUST_GDA2020.shp")
postcodes = pd.read_csv("../data/raw/external/postcode.csv", names = ["POA_CODE21", "Name", "Area"])
postcodes["POA_CODE21"] = postcodes["POA_CODE21"].astype(int)

# Converting the geometry shaape to to latitude and longitude
# TAKEN FROM TUTE 2 NOTEBOOK
sf["geometry"] = sf["geometry"].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sf = sf[sf["POA_CODE21"].between("3000", "4000")]
sf["POA_CODE21"] = sf["POA_CODE21"].astype(int)

gdf = gpd.GeoDataFrame(
    pd.merge(postcodes, sf, on = "POA_CODE21", how = "inner")
)

geoJSON = gdf[["POA_CODE21", "geometry"]].drop_duplicates("POA_CODE21").to_json()

In [48]:
# (y, x) since we want (lat, long)
gdf["centroid"] = gdf["geometry"].apply(lambda x: (x.centroid.y, x.centroid.x))
gdf[["Name", "POA_CODE21", "centroid"]].head()

affordability_df.rename({"Postcode": "POA_CODE21"})

m = folium.Map(location = [-37.8136, 144.9631], tiles = "Stamen Terrain", zoom_start = 10)

c = folium.Choropleth(
            geo_data = geoJSON, # geoJSON 
            name = "choropleth", # name of plot
            data = affordability_df, # data source
            columns = ["Postcode", "Standardised Affordability"], # the columns required
            key_on = "properties.POA_CODE21", # this is from the geoJSON's properties
            fill_color = "YlOrRd", # color scheme
            nan_fill_color = "grey",
            legend_name = "Affordability"
        )

c.add_to(m)
m.save(f"../plots/affordability_heatmap")

In [52]:
# Creating GeoPandas visualisation of top 10 most affordable postcodes
affordability_df.to_csv("../data/curated/affordability.csv", index=False)

postcodes["POA_CODE21"] = postcodes["POA_CODE21"].astype(int)
postcodes = postcodes.loc[postcodes["POA_CODE21"].isin(affordability_df["Postcode"].to_list())]

affordability_df.sort_values(by="Standardised Affordability", ascending=False).head(10)


,Postcode,Standardised Affordability
238,3424,1.000000
235,3409,0.947093
236,3414,0.946619
237,3418,0.942966
4,3006,0.936327
233,3396,0.929125
232,3393,0.909012
259,3490,0.906518
231,3390,0.905940
211,3318,0.901626


In [54]:
gdf = gpd.GeoDataFrame(
    pd.merge(postcodes, sf, on = "POA_CODE21", how = "inner")
)

geoJSON = gdf[["POA_CODE21", "geometry"]].drop_duplicates("POA_CODE21").to_json()

m = folium.Map(location = [-37.8136, 144.9631], tiles = "Stamen Terrain", zoom_start = 10)

m.add_child(folium.Choropleth(geo_data = geoJSON, name = "choropleth",))
m.save(f"../plots/most_affordable_heatmap")